In [42]:
# Import Libraries
import pandas as pd
import pickle as pk
import numpy as np
from collections import defaultdict
import time

In [43]:
# Import processed sample data from folder
infile = open("Stored Data/sample_review_w_aspects.pickle", "rb")
data_reviews = pk.load(infile)
infile.close()

data_reviews = data_reviews.dropna(axis="rows", how="any")
data_reviews.head()

,Hotel,Reviews,Date,Score,Aspect,Sentiment
0,Hotel Arena,Only the park outside of the hotel was beauti...,8/3/2017,2.9,"[(""beautiful"",""park"")]","{'park': [0.75, 0]}"
1,Hotel Arena,No real complaints the hotel was great great ...,8/3/2017,7.5,"[(""real"",""complaints""),(""surroundings"",""compla...","{'food': [1.0, 0], 'spot': [0.375, 0], 'staff'..."
2,K K Hotel George,Very comfortable beds smart bathroom good sho...,8/3/2017,9.6,"[(""very comfortable"",""beds""),(""smart"",""bathroo...","{'staff': [0.125, -0.25], 'shower': [0.5, 0], ..."
3,Apex Temple Court Hotel,Everything,8/3/2017,10.0,[],{}
4,Apex Temple Court Hotel,Great Hotel with wonderful staff and good cui...,8/3/2017,7.9,"[(""wonderful"",""staff""),(""good"",""cuisine"")]","{'cuisine': [0.5, 0], 'staff': [0.75, 0]}"


In [44]:
infile = open("Stored Data/sample_hotels.pickle", "rb")
data_hotels = pk.load(infile)
infile.close()

data_hotels.head()

,Hotel,Address,Score,Latitude,Longitude,City
0,Hotel Arena,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,52.360576,4.915968,"Amsterdam, Netherlands"
1,K K Hotel George,1 15 Templeton Place Earl s Court Kensington a...,8.5,51.491888,-0.194971,"London, United Kingdom"
2,Apex Temple Court Hotel,1 2 Serjeant s Inn Fleet Street City of London...,9.2,51.513734,-0.108751,"London, United Kingdom"
3,The Park Grand London Paddington,1 3 Queens Garden Westminster Borough London W...,7.7,51.514218,-0.180903,"London, United Kingdom"
4,Park Plaza County Hall London,1 Addington Street Lambeth London SE1 7RY Unit...,8.4,51.501400,-0.116009,"London, United Kingdom"


In [45]:
hotel_dict = {}

for row in data_hotels.itertuples():
    hotel_dict.setdefault(row[1], [row[2],row[3],row[4],row[5],row[6]])

hotel_dict

{'Radisson Blu Hotel Milan': ['Via Villapizzone 24 Certosa 20156 Milan Italy',
  8.1999999999999993,
  45.500667,
  9.1445874000000007,
  'Another City'],
 'UNA Hotel Mediterraneo': ['Via L Muratori 14 Porta Romana 20135 Milan Italy',
  8.1999999999999993,
  45.451872999999999,
  9.2054799999999997,
  'Another City'],
 'Grupotel Gran Via 678': ['Gran Via de les Corts Catalanes 678 Eixample 08010 Barcelona Spain',
  8.4000000000000004,
  41.392344000000001,
  2.1729157000000003,
  'Barcelona. Spain'],
 'Medinaceli': ['Duc De Medinaceli 8 Ciutat Vella 08002 Barcelona Spain',
  7.7999999999999998,
  41.378168500000001,
  2.1786626999999998,
  'Barcelona. Spain'],
 'Baglioni Hotel Carlton The Leading Hotels of the World': ['Via Senato 5 Milan City Center 20121 Milan Italy',
  8.8000000000000007,
  45.469166299999998,
  9.1984454000000007,
  'Another City'],
 'Apex City Of London Hotel': ['No 1 Seething Lane City of London London EC3N 4AX United Kingdom',
  9.0,
  51.510462400000002,
  -0.0

In [51]:
def get_city(hotel_dict):
    city_dict = {}
    
    for key, value in hotel_dict.items():
        if value[4] in city_dict:
            city_dict[value[4]] += 1
        else:
            city_dict.setdefault(value[4], 0)
        
    return city_dict

city_dict = get_city(hotel_dict)
city_dict

{'Amsterdam, Netherlands': 100,
 'Another City': 115,
 'Barcelona. Spain': 179,
 'London, United Kingdom': 359,
 'Paris, France': 273,
 'Vienna, Austria': 121}

In [55]:
def timeline(data_reviews):
    timeline_dict = {}
    aspect_dict = {}
    
    for row in data_reviews.itertuples():
        for key, value in row[6].items():
            if key in aspect_dict:
                aspect_dict[key] += 1
            else:
                aspect_dict.setdefault(key, 1)

            
            if row[3] in timeline_dict:
                if hotel_dict[row[1]][4] in timeline_dict[row[3]]:
                    if key in timeline_dict[row[3]][hotel_dict[row[1]][4]]:
                        timeline_dict[row[3]][hotel_dict[row[1]][4]][key].append([row[1], (value[0]+value[1])])
                    else:
                        timeline_dict[row[3]][hotel_dict[row[1]][4]].setdefault(key, [[row[1], (value[0]+value[1])]])
                else:
                    timeline_dict[row[3]].setdefault(hotel_dict[row[1]][4], {key: [[row[1], (value[0]+value[1])]]})
            else:
                timeline_dict.setdefault(row[3], {hotel_dict[row[1]][4]: {key: [[row[1], (value[0]+value[1])]]} })
                
    return timeline_dict, aspect_dict
        
overall = timeline(data_reviews)
timeline_dict = overall[0]
aspect_dict = overall[1]

In [56]:
aspect_dict

{'Toothbrush': 1,
 'Thriller': 1,
 'Reception': 41,
 'Toilet': 8,
 'Grit': 1,
 'night': 130,
 's.': 3,
 'royalty': 1,
 'plaza': 1,
 'yea': 1,
 'linen': 10,
 'dress': 2,
 'days': 33,
 'reason': 11,
 'WC': 4,
 'Sofa': 1,
 'walkways': 2,
 'availability': 2,
 'boards': 1,
 'memory': 1,
 'Spread': 1,
 'living': 7,
 'executive': 25,
 'odour': 2,
 'kind': 16,
 'waitstaff': 1,
 'que': 1,
 'commodation': 1,
 'cyclist': 1,
 'THey': 1,
 'parking': 108,
 'quaintness': 1,
 'knee': 1,
 'incl': 2,
 'crown': 1,
 'husbands': 4,
 'privilege': 3,
 'maids': 1,
 'network': 5,
 'station': 129,
 'cards': 12,
 'Pasta': 1,
 'lengths': 2,
 'notices': 1,
 'shadow': 1,
 'percent': 2,
 'hustle': 1,
 'nighbourhood': 1,
 'passageways': 1,
 'sharing': 1,
 'choices': 26,
 'aspects': 2,
 'kinds': 1,
 'marterial': 1,
 'fresheners': 1,
 'traffic': 14,
 'vehicle': 1,
 'Ps': 1,
 'plazas': 1,
 'LAdy': 1,
 'width': 1,
 'Roms': 1,
 'ibis': 1,
 'hospital': 2,
 'themes': 1,
 'workers': 2,
 'penthouse': 1,
 'improvements': 1,
 '

In [57]:
timeline_dict

{'7/29/2017': {'Amsterdam, Netherlands': {'11am': [['INK Hotel Amsterdam MGallery by Sofitel',
     0]],
   'Amsterdam': [['WestCord Fashion Hotel Amsterdam', 0]],
   'Bed': [['Best Western Delphi Hotel', -0.75]],
   'Breakfast': [['citizenM Amsterdam', 0],
    ['INK Hotel Amsterdam MGallery by Sofitel', 0]],
   'Cleanliness': [['DoubleTree by Hilton Amsterdam Centraal Station', 0]],
   'Closing': [['art otel Amsterdam', 0]],
   'Dam': [['art otel Amsterdam', 0]],
   'Front': [['Bilderberg Hotel Jan Luyken', 0]],
   'Im': [['WestCord Fashion Hotel Amsterdam', 0]],
   'Kids': [['citizenM Amsterdam', 0]],
   'Nothing': [['Corendon Vitality Hotel Amsterdam', -0.625],
    ['Hotel V Nesplein', 0.125],
    ['Luxury Suites Amsterdam', -0.25]],
   'Room': [['Jaz Amsterdam', 0],
    ['citizenM Amsterdam', 0],
    ['Best Western Delphi Hotel', 0],
    ['Mercure Hotel Amsterdam West', 0]],
   'Rooms': [['art otel Amsterdam', 0]],
   'Shower': [['Best Western Delphi Hotel', 0],
    ['Grand Hotel A

In [59]:
# Store the hotel data
outfile = open("Stored Data/hotel_dict.pickle", "wb")
pk.dump(hotel_dict, outfile)
outfile.close()

# Store the city data
outfile = open("Stored Data/city_dict.pickle", "wb")
pk.dump(city_dict, outfile)
outfile.close()

# Store the aspect data
outfile = open("Stored Data/aspect_dict.pickle", "wb")
pk.dump(aspect_dict, outfile)
outfile.close()

# Store the timeline data
outfile = open("Stored Data/timeline_dict.pickle", "wb")
pk.dump(timeline_dict, outfile)
outfile.close()